In [ ]:
# ================================
# 1. Import Required Libraries
# ================================
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os
import zipfile

# ================================
# 2. Paths for Dataset
# ================================
zip_file_path = "Dataset.zip"
extracted_folder = "Dataset"

# ================================
# 3. Extract ZIP if not extracted
# ================================
if not os.path.exists(extracted_folder):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")

# ================================
# 4. Image Settings
# ================================
img_height, img_width = 224, 224
batch_size = 8
AUTOTUNE = tf.data.AUTOTUNE

# ================================
# 5. Load Dataset
# ================================
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(extracted_folder, "train"),
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(extracted_folder, "val"),
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(extracted_folder, "test"),
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

# Improve performance (prefetching)
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

# ================================
# 6. Calculate Class Weights
# ================================
train_labels = []
for _, labels in train_ds.unbatch():
    train_labels.append(int(labels.numpy()))

class_weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_labels),
    y=train_labels
)

class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("\n🔢 Class Weights:", class_weights_dict)

# ================================
# 7. Data Augmentation Layer
# ================================
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# ================================
# 8. Build CNN Model
# ================================
model = models.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dropout(0.5),

    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# ================================
# 9. Compile Model
# ================================
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ================================
# 10. Early Stopping Callback
# ================================
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# ================================
# 11. Train Model
# ================================
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping],
    class_weight=class_weights_dict
)

# ================================
# 12. Evaluate on Test Set
# ================================
test_loss, test_acc = model.evaluate(test_ds)
print("\n✅ Test Accuracy:", round(test_acc * 100, 2), "%")

# ================================
# 13. Save Model
# ================================
model.save("pneumonia_model.h5")

# ================================
# 14. Plot Training Curves
# ================================
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training & Validation Accuracy')
plt.grid()
plt.show()

# ================================
# 15. Predictions on Test Data
# ================================
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend([1 if p > 0.5 else 0 for p in preds])

# ================================
# 16. Evaluation Reports
# ================================
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\n📄 Classification Report:")
print(classification_report(y_true, y_pred, target_names=["Normal", "Pneumonia"]))
